In [1]:
import torch
import os
import gzip
import pickle
import time
import numpy as np 
import tensorflow as tf

2023-04-30 14:29:23.586014: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load and examine dataset

In [2]:
""" Load the dataset
Code adapted from http://deeplearning.net/tutorial/code/logistic_sgd.py

:type dataset: string
:param dataset: the path to the dataset (here MNIST)
"""
# Download the MNIST dataset if it is not present
dataset = "mnist.pkl.gz"

data_dir, data_file = os.path.split(dataset)
if data_dir == "" and not os.path.isfile(dataset):
    # Check if dataset is in the data directory.
    new_path = os.path.join(
        os.path.split(__file__)[0],
        dataset
    )
    if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
        dataset = new_path

if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
    from six.moves import urllib
    origin = (
        'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
    )
    print('Downloading data from %s' % origin)
    urllib.request.urlretrieve(origin, dataset)

print('Loading data...')

# Load the dataset
with gzip.open(dataset, 'rb') as f:
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set = pickle.load(f)

print('Loaded train_set, valid_set and test_set.')

Loading data...
Loaded train_set, valid_set and test_set.


In [3]:
batch_size = 64

train_set = (train_set[0].astype('float32'), train_set[1].astype('int64'))
valid_set = (valid_set[0].astype('float32'), valid_set[1].astype('int64'))
test_set = (test_set[0].astype('float32'), test_set[1].astype('int64'))

train_dataset = tf.data.Dataset.from_tensor_slices(train_set)
train_loader = train_dataset.shuffle(buffer_size=10000).batch(batch_size)
valid_dataset = tf.data.Dataset.from_tensor_slices(valid_set)
valid_loader = valid_dataset.batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices(test_set)
test_loader = test_dataset.batch(batch_size)


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(
    optimizer=tf.keras.optimizers.SGD(0.05),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    train_loader,
    epochs=20,
    validation_data=valid_loader,
    # output epoch 0 as well
    initial_epoch=0,
)

Epoch 1/20


2023-04-30 14:29:29.421814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [50000,784]
	 [[{{node Placeholder/_0}}]]
2023-04-30 14:29:29.422108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [50000]
	 [[{{node Placeholder/_1}}]]


780/782 [============================>.] - ETA: 0s - loss: 0.4440 - sparse_categorical_accuracy: 0.8779

2023-04-30 14:29:31.694349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [10000]
	 [[{{node Placeholder/_1}}]]


782/782 [==============================] - 3s 3ms/step - loss: 0.4439 - sparse_categorical_accuracy: 0.8779 - val_loss: 0.3309 - val_sparse_categorical_accuracy: 0.8987
Epoch 2/20
782/782 [==============================] - 2s 3ms/step - loss: 0.2151 - sparse_categorical_accuracy: 0.9385 - val_loss: 0.2241 - val_sparse_categorical_accuracy: 0.9338
Epoch 3/20
782/782 [==============================] - 2s 3ms/step - loss: 0.1606 - sparse_categorical_accuracy: 0.9534 - val_loss: 0.1390 - val_sparse_categorical_accuracy: 0.9618
Epoch 4/20
782/782 [==============================] - 2s 3ms/step - loss: 0.1268 - sparse_categorical_accuracy: 0.9642 - val_loss: 0.1846 - val_sparse_categorical_accuracy: 0.9397
Epoch 5/20
782/782 [==============================] - 2s 3ms/step - loss: 0.1038 - sparse_categorical_accuracy: 0.9701 - val_loss: 0.1145 - val_sparse_categorical_accuracy: 0.9657
Epoch 6/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0883 - sparse_categorical_accuracy:

In [ ]:
import json

# save the results: train_loss_list, train_acc_list, valid_acc_list
train_loss_list = model.history.history['loss']
train_acc_list = model.history.history['sparse_categorical_accuracy']
valid_acc_list = model.history.history['val_sparse_categorical_accuracy']

with open("results/tensorflow_mlp.json", "w") as f:
    json.dump({
        "train_loss": train_loss_list,
        "train_acc": train_acc_list,
        "valid_acc": valid_acc_list
    }, f)